In [ ]:
############# GSR Proccessig and Feature Extraction ############## #Takes about 23 minutes#
import pandas as pd
import os
import neurokit2 as nk
import matplotlib.pyplot as plt
import glob
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve
import seaborn as sns
from sklearn.impute import SimpleImputer

# Directory paths
block_details_dir = '/Users/cahree/Desktop/REU-2024-Cognitive-Load/CLAS_Database/CLAS/Block_details/'
block_gsr_ppg_data_path = '/Users/cahree/Desktop/REU-2024-Cognitive-Load/CLAS_Database/CLAS/Participants/'

# Get all block details CSV files
all_users_block_details_paths = glob.glob(os.path.join(block_details_dir, '*.csv'))

# Blocks of interest
blocks_of_interest = [1, 2, 4, 8, 10]

# Cognitive load labels based on block numbers
cognitive_load_labels = {1: 1, 4: 1, 10: 1, 2: 2, 8: 2}
# Initialize lists to store features and labels
features = []
labels = []
count = 0

# Iterate over each block details file
for block_details_path in all_users_block_details_paths:
    #if count == 3:
        #break
    print(count)
    count+=1
    block_details = pd.read_csv(block_details_path)
    filtered_block_details = block_details[block_details['Block'].isin(blocks_of_interest)]
    sorted_filtered_block_details = filtered_block_details.sort_values(by='Block')

    user_id = os.path.basename(block_details_path).split('_')[0]

    # Iterate through each row in block details to load corresponding PPG files
    for _, row in sorted_filtered_block_details.iterrows():
        block = row['Block']
        ppg_file = row['EDA&PPG File']

        # Determine the correct filename for special blocks
        if block == 2:
            type = 'mathtest'
            base, ext = ppg_file.rsplit('.', 1)
            new_filename = f"{base}{type}.{ext}"
        elif block == 8:
            type = 'IQtest'
            base, ext = ppg_file.rsplit('.', 1)
            new_filename = f"{base}{type}.{ext}"
        else: 
            new_filename = ppg_file
        
        file_path = os.path.join(block_gsr_ppg_data_path, user_id, 'by_block', new_filename)
        
        try:
            # Load the PPG and GSR data
            block_data = pd.read_csv(file_path)
            gsr_data = block_data['gsr']

            # Process and clean GSR data
            gsr_data_cleaned = nk.eda_clean(gsr_data, sampling_rate=256, method="BioSPPy")
            cvxEDA = nk.eda_phasic(gsr_data_cleaned, sampling_rate=256, method="cvxEDA")

            # Extract tonic and phasic components
            t1 = cvxEDA["EDA_Tonic"].values
            p1 = cvxEDA["EDA_Phasic"].values

            # Calculate the mean EDA from the cleaned signal
            mean_eda = gsr_data_cleaned.mean()
    

            # Extract EDA peaks
            _, eda_features = nk.eda_peaks(p1, sampling_rate=256, method="neurokit")
            # print(eda_features.keys())

            # Convert the extracted features to a DataFrame
            eda_features_df = pd.DataFrame({
                'SCR_Onsets': eda_features['SCR_Onsets'],
                'SCR_Peaks': eda_features['SCR_Peaks'],
                'SCR_Height': eda_features['SCR_Height'],
                'SCR_Amplitude': eda_features['SCR_Amplitude'],
                'SCR_RiseTime': eda_features['SCR_RiseTime'],
                'SCR_Recovery': eda_features['SCR_Recovery'],
                'SCR_RecoveryTime': eda_features['SCR_RecoveryTime'],
                'mean_eda':  mean_eda
            })

            # Handle missing values by imputing
            eda_features_df = eda_features_df.fillna(0)

            # Append each feature set with its corresponding label
            for _, feature_set in eda_features_df.iterrows():
                features.append(feature_set)
                labels.append(cognitive_load_labels[block])

        except Exception as e:
            print(f"Error loading or processing PPG data for User {user_id}, Block {block}: {e}")

print(len(features))
print(len(labels))

features_df = pd.DataFrame(features)
display(features_df)
labels_df = pd.DataFrame(labels, columns=['Cognitive_Load_Label'])

save_dir = '/Users/cahree/Desktop/REU-2024-Cognitive-Load'

# Convert features and labels lists to DataFrames
features_df = pd.DataFrame(features)
labels_df = pd.DataFrame(labels, columns=['Cognitive_Load_Label'])

# Reset indices of features and labels DataFrames
features_df = features_df.reset_index(drop=True)
labels_df = labels_df.reset_index(drop=True)

# Optionally, concatenate the features and labels into a single DataFrame
combined_df = pd.concat([features_df, labels_df], axis=1)

# Define full paths for each CSV file
features_path = os.path.join(save_dir, 'GSRfeatures.csv')
labels_path = os.path.join(save_dir, 'GSRlabels.csv')
combined_path = os.path.join(save_dir, 'GSRfeatures_with_labels.csv')

# Save the DataFrames to the specified paths
features_df.to_csv(features_path, index=False)
labels_df.to_csv(labels_path, index=False)
combined_df.to_csv(combined_path, index=False)




In [ ]:
######### PPG Proccessig and Feature Extraction ##########  #Takes abut 3 min #


import pandas as pd
import os
import neurokit2 as nk
import matplotlib.pyplot as plt
import glob
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve
import seaborn as sns
from sklearn.impute import SimpleImputer

# Directory paths
block_details_dir = '/Users/cahree/Desktop/REU-2024-Cognitive-Load/CLAS_Database/CLAS/Block_details/'
block_gsr_ppg_data_path = '/Users/cahree/Desktop/REU-2024-Cognitive-Load/CLAS_Database/CLAS/Participants/'

# Get all block details CSV files
all_users_block_details_paths = glob.glob(os.path.join(block_details_dir, '*.csv'))

# Blocks of interest
blocks_of_interest = [1, 2, 4, 8, 10]

# Cognitive load labels based on block numbers
cognitive_load_labels = {1: 1, 4: 1, 10: 1, 2: 2, 8: 2}

# Initialize lists to store features and labels
features = []
labels = []
count = 0

# Iterate over each block details file
for block_details_path in all_users_block_details_paths:
    #if count == 3:
       # break
    count += 1
    print(count)
    block_details = pd.read_csv(block_details_path)
    filtered_block_details = block_details[block_details['Block'].isin(blocks_of_interest)]
    sorted_filtered_block_details = filtered_block_details.sort_values(by='Block')

    user_id = os.path.basename(block_details_path).split('_')[0]

    # Iterate through each row in block details to load corresponding PPG files
    for _, row in sorted_filtered_block_details.iterrows():
        block = row['Block']
        ppg_file = row['EDA&PPG File']

        if block == 2:
            type = 'mathtest'
            base, ext = ppg_file.rsplit('.', 1)
            new_filename = f"{base}{type}.{ext}"
            file_path = os.path.join(block_gsr_ppg_data_path, user_id, 'by_block', new_filename)
            
        elif block == 8:
            type = 'IQtest'
            base, ext = ppg_file.rsplit('.', 1)
            new_filename = f"{base}{type}.{ext}"
            file_path = os.path.join(block_gsr_ppg_data_path, user_id, 'by_block', new_filename)
            
        else: 
            file_path = os.path.join(block_gsr_ppg_data_path, user_id, 'by_block', ppg_file)

        print(f"Loading PPG data for User {user_id}, Block {block} from: {file_path}")
        
        # Load the PPG and GSR data
        try:
            block_data = pd.read_csv(file_path)
            # Extract PPG and GSR columns
            ppg_data = block_data['ppg']

            # Process the PPG data
            ppg_signals, info = nk.ppg_process(ppg_data.values.flatten(), sampling_rate=256)
            
            # Analyze the PPG data using interval-related analysis
            ppg_analysis = nk.ppg_analyze(ppg_signals, sampling_rate=256, method='interval-related')

            # Extract features of interest
            ppg_features_df = pd.DataFrame({
                'HRV_RMSSD': ppg_analysis['HRV_RMSSD'],
                'HRV_MeanNN': ppg_analysis['HRV_MeanNN'],
                'HRV_SDNN': ppg_analysis['HRV_SDNN'],
                'HRV_ApEn': ppg_analysis['HRV_ApEn'],
                'PPG_Clean_Mean': ppg_signals['PPG_Clean'].mean()
            })


            # Append each feature set with its corresponding label
            for _, feature_set in ppg_features_df.iterrows():
                features.append(feature_set)
                labels.append(cognitive_load_labels[block])

        except Exception as e:
            print(f"Error loading or processing PPG data for User {user_id}, Block {block}: {e}")


print(len(features))
print(len(labels))

features_df = pd.DataFrame(features)
display(features_df)
labels_df = pd.DataFrame(labels, columns=['Cognitive_Load_Label'])

save_dir = '/Users/cahree/Desktop/REU-2024-Cognitive-Load'

# Convert features and labels lists to DataFrames
features_df = pd.DataFrame(features)
labels_df = pd.DataFrame(labels, columns=['Cognitive_Load_Label'])

# Reset indices of features and labels DataFrames
features_df = features_df.reset_index(drop=True)
labels_df = labels_df.reset_index(drop=True)

combined_df = pd.concat([features_df, labels_df], axis=1)

# paths for each CSV file
features_path = os.path.join(save_dir, 'PPGfeatures.csv')
labels_path = os.path.join(save_dir, 'PPGlabels.csv')
combined_path = os.path.join(save_dir, 'PPGfeatures_with_labels.csv')

# Save the DataFrames paths
features_df.to_csv(features_path, index=False)
labels_df.to_csv(labels_path, index=False)
combined_df.to_csv(combined_path, index=False)


In [ ]:
################# EEG Proccessig and Feature Extraction ################### # Takes about 13 minutes #

import numpy as np
import pandas as pd
from glob import glob
#import EEGEXTRACT2 as eeg
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

fs = 256

def extract_user_id(file_name):
    return file_name.split('/')[-1]
    
def extract_level_number(file_path):
    # Extract the filename from the file path
    filename = file_path.split('/')[-1]
    
    # Remove the .csv extension
    base_name = filename.replace('.csv', '')
    
    # Find the last underscore and extract the number
    number_str = base_name.split('_')[-1]
    
    try:
        return int(number_str)
    except ValueError:
        raise ValueError("No valid number found in filename")

def extract_all_levels_files(path):
    all_scenario_files = glob(path + '/*data_level*')
    all_scenario_files.sort(key=extract_user_id)
    return all_scenario_files
    
def load_labels(user_id, label_path='/Users/cahree/Desktop/REU-2024-Cognitive-Load/CL_Drive_Database/Labels'):
    """Load the labels for a specific user based on user ID."""
    label_file = f"{label_path}/{user_id}.csv"  # Adjust the pattern based on actual file names
    return pd.read_csv(label_file)
    
def extract_all_user_eeg_files(path):
    all_user_eeg_files = glob(path + '/*')
    return all_user_eeg_files

def normalize_to_custom_range(data, new_min, new_max):
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val) * (new_max - new_min) + new_min


def segment_data_func(file):
    removed_segments = []
    # print("running segment_data_func")
    
    # Load the CSV data
    df = pd.read_csv(file)
    
    # Check for NaNs in the entire file
    if df.isna().sum().sum() > 0:
        print(f"File {file} contains NaNs and will be skipped.")
        return None
    
    # Define the known sampling frequency
    sfreq = 256  # Sampling frequency in Hz
    
    # Calculate the number of samples per 10-second segment
    segment_length = int(10 * sfreq)
    
    # Check total length of data
    total_samples = len(df)
    
    # Ensure we can create exactly 18 segments
    if total_samples < 18 * segment_length:
        print(f"Warning: The data file {file} does not contain enough samples for 18 segments of 10 seconds each.")
        return None
    
    # Initialize a list to store all segments
    segmented_data = []
    
    # Segment the data into 10-second intervals
    n_segments = 18
    for i in range(n_segments):
        start_sample = i * segment_length
        end_sample = start_sample + segment_length
        # Extract the segment
        segment = df.iloc[start_sample:end_sample].copy()
        # Add a segment identifier column
        segment.loc[:, 'segment'] = i
        # Append the segment to the list
        segmented_data.append(segment)
    
    # Combine all segments into a single DataFrame
    segmented_df = pd.concat(segmented_data, ignore_index=True)
    
    # Extract the channel columns (assuming channel columns are numeric and are not 'segment')
    channels = [col for col in segmented_df.columns if col != 'segment' and col !='Timestamp']
    # Determine the number of channels
    n_channels = len(channels)
    
    # Prepare data for reshaping
    reshaped_data = np.empty((n_channels, segment_length, len(segmented_data)))
    for i, segment in enumerate(segmented_data):
        for j, channel in enumerate(channels):
            reshaped_data[j, :, i] = segment[channel].values
            
    # print(reshaped_data)
    return reshaped_data, removed_segments

def aggregate_connectivity_features(connectivity_features, epochs):
    print("connectivity_features shape:", connectivity_features.shape)
    # Adjust according to the correct shape
    # Assuming 6 pairs as you have (18, 6), modify aggregation logic as needed
    num_channels = 4
    num_pairs = num_channels * (num_channels - 1) // 2
    try:
        aggregated_features = np.mean(connectivity_features.reshape(-1, num_pairs, epochs), axis=1)
    except ValueError as e:
       #  print("Reshape failed:", e)
        # print("Connectivity features shape:", connectivity_features.shape)
        # print("Expected reshape dimensions:", num_pairs, epochs)
        raise
    return aggregated_features



def feature_extraction(eegData, bin_min, bin_max, binWidth):
    n_channels = eegData.shape[0]
    # print(" n_channels: ",  n_channels)
    epochs = eegData.shape[2]
    features = []
    feature_names = []

    ShannonRes = eeg.shannonEntropy(eegData, bin_min, bin_max, binWidth)
    ShannonRes = ShannonRes.T
    features.append(ShannonRes)
    # print("ShannonRes", ShannonRes.shape)
    feature_names.extend([f"ShannonEntropy_{i}" for i in range(ShannonRes.shape[1])])

    medianFreqRes = eeg.medianFreq(eegData, fs)
    medianFreqRes = medianFreqRes.T
    features.append(medianFreqRes)
    feature_names.extend([f"MedianFreq_{i}" for i in range(medianFreqRes.shape[1])])

    std_res = eeg.eegStd(eegData)
    std_res = std_res.T
    features.append(std_res)
    feature_names.extend([f"Std_{i}" for i in range(std_res.shape[1])])

    subbands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    for band in subbands:
        freq_bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100)}
        eegData_band = eeg.filt_data(eegData, *freq_bands[band], fs)
        ShannonRes_band = eeg.shannonEntropy(eegData_band, bin_min, bin_max, binWidth)
        ShannonRes_band = ShannonRes_band.T
        features.append(ShannonRes_band)
        feature_names.extend([f"ShannonEntropy_{band}_{i}" for i in range(ShannonRes_band.shape[1])])

    HjorthMob, HjorthComp = eeg.hjorthParameters(eegData)
    HjorthMob = HjorthMob.T
    HjorthComp = HjorthComp.T
    features.append(HjorthMob)
    features.append(HjorthComp)
    feature_names.extend([f"HjorthMob_{i}" for i in range(HjorthMob.shape[1])])
    feature_names.extend([f"HjorthComp_{i}" for i in range(HjorthComp.shape[1])])

    bands = ['alpha', 'beta', 'gamma']
    for band in bands:
        freq_bands = {'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100)}
        bandPwr = eeg.bandPower(eegData, *freq_bands[band], fs)
        bandPwr = bandPwr.T
        features.append(bandPwr)
        feature_names.extend([f"BandPower_{band}_{i}" for i in range(bandPwr.shape[1])])
    mi_features = []
    for ch1 in range(n_channels):
        for ch2 in range(ch1 + 1, n_channels):
            mutual_info = eeg.calculate2Chan_MI(eegData, ch1, ch2, bin_min, bin_max, binWidth)
            if mutual_info.ndim == 1:
                mutual_info = mutual_info[:, np.newaxis]
            mi_features.append(mutual_info)
    if mi_features:
        mi_features = np.concatenate(mi_features, axis=1)
        # print("mi_features shape before aggregation:", mi_features.shape)
        mi_features = aggregate_connectivity_features(mi_features, epochs)
        mi_features = mi_features.T
        features.append(mi_features)
        feature_names.extend([f"MI_{i}" for i in range(mi_features.shape[1])])

    pli_features = []
    for ch1 in range(n_channels):
        for ch2 in range(ch1 + 1, n_channels):
            pli = eeg.phaseLagIndex(eegData, ch1, ch2)
            if pli.ndim == 1:
                pli = pli[:, np.newaxis]
            pli_features.append(pli)
    if pli_features:
        pli_features = np.concatenate(pli_features, axis=1)
        pli_features = aggregate_connectivity_features(pli_features, epochs)
        pli_features = pli_features.T
        features.append(pli_features)
        feature_names.extend([f"PLI_{i}" for i in range(pli_features.shape[1])])

    try:
        concatenated_features = np.concatenate([f for f in features], axis=1)
        print("Concatenated Features Shape:", concatenated_features.shape)
    except ValueError as e:
        print(f"Error in concatenating features: {e}")
        for i, f in enumerate(features):
            print(f"Feature {i} shape: {f.shape}")
        raise

    return concatenated_features, feature_names


  